In [ ]:
import numpy as np
import h5py
from scipy import signal
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import glob
import os
import mne
from eeghdf import EEGHDFUpdater
root_path = "//172.16.88.200/private/2221012/MIOnly_FTP_EEG Dataset and OpenBMI Toolbox for Three BCI Paradigms"
file_paths = glob.glob(root_path + "/pres/*.set")
fs = 500
ch = 14 #C4

In [ ]:
finfo_list = []
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    fns = file_name.split("_")
    session = int(fns[0][4:])
    subject = int(fns[1][4:])
    finfo_list.append((file_path,session,subject))

'Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'FC3', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CPz', 'CP4', 'P1', 'P2', 'POz', 'FT9', 'FTT9h', 'TTP7h', 'TP7', 'TPP9h', 'FT10', 'FTT10h', 'TPP8h', 'TP8', 'TPP10h', 'F9', 'F10', 'AF7', 'AF3', 'AF4', 'AF8', 'PO3', 'PO4'

In [ ]:
eeglist = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10', 'FC3', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CPz', 'CP4', 'P1', 'P2', 'POz', 'FT9', 'FTT9h', 'TTP7h', 'TP7', 'TPP9h', 'FT10', 'FTT10h', 'TPP8h', 'TP8', 'TPP10h', 'F9', 'F10', 'AF7', 'AF3', 'AF4', 'AF8', 'PO3', 'PO4']
using_lst = ['C3','Cz','C4','C1','C2','FC1','FC5','FC2','FC6','CP1','CP5','CP2','CP6']
ch_indexes = []
for item in using_lst:
    ch_indexes.append(eeglist.index(item))

len(ch_indexes),ch_indexes

## Preview

In [ ]:
preview_path,session,subject, = finfo_list[6]
x = mne.io.read_epochs_eeglab(preview_path).get_data(item=["left","right"])
x.shape,session,subject

### 標準化

In [ ]:
dx = x[0,:,:]
dx = StandardScaler().fit_transform(dx.T).T
#test: np.array([[1,2,3],[10,20,30]]),StandardScaler().fit_transform(np.array([[1,2,3],[10,20,30]]).T).T

### パワースペクトルに変換

In [ ]:
# パワースペクトルへの変換
freq, px = signal.periodogram(dx, fs=fs)
trimmed_freq_flags = (freq >= 8) & (freq <= 20)
print(len(freq[trimmed_freq_flags]),np.where(trimmed_freq_flags)[0][0],np.where(trimmed_freq_flags)[0][-1])
plt.plot(freq[trimmed_freq_flags],np.mean(px,axis=0)[trimmed_freq_flags])
plt.ylim((0,0.050))

## まとめて処理(trainのみ)

In [ ]:
updater = EEGHDFUpdater(hdf_path=root_path+"/3pdataset.h5",
                        fs=fs,
                        lables=["left","right"])
updater.remove_hdf()
for path,session,subject in finfo_list:
    updater.add_eeglab(path,{"session":int(session),"subject":int(subject)})

In [ ]:
from scipy.signal import butter, filtfilt
updater = EEGHDFUpdater(hdf_path=root_path+"/3pdataset.h5",
                        fs=fs,
                        lables=["left","right"])
def bandpass(data):
    lowcut = 8  # バンドパスフィルタの下限周波数
    highcut = 20  # バンドパスフィルタの上限周波数
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(2, [low, high], btype='band')
    data = filtfilt(b, a, data)
    return data
#std
def prepro_func(x:np.ndarray):
    x = bandpass(x)
    return StandardScaler().fit_transform(x.T).T #標準化
updater.preprocess("std",prepro_func)

#psd
def prepro_func(x:np.ndarray):
    x = bandpass(x)
    x = StandardScaler().fit_transform(x.T).T
    _, px = signal.periodogram(x, fs=fs)
    return px #note:31:81
updater.preprocess("psd",prepro_func)